In [2]:
from lenskit.datasets import ML100K
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn

import pandas as pd

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

In [5]:
foursquare_clean_DIR = './data/clean/foursquare'
gowalla_clean_DIR = './data/clean/gowalla'
ml100_clean_DIR = './data/clean/ml-100k'
ml25_clean_DIR = './data/clean/ml-25m'

In [11]:
train = pd.read_csv(f"{foursquare_clean_DIR}/train.tsv", sep='\t', names=['user', 'item'])
tune = pd.read_csv(f"{foursquare_clean_DIR}/tune.tsv", sep='\t', names=['user', 'item'])
test = pd.read_csv(f"{foursquare_clean_DIR}/test.tsv", sep='\t', names=['user', 'item'])

In [18]:
train['rating'] = 1
tune['rating'] = 1
test['rating'] = 1

In [14]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [15]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [21]:
all_recs = []
test_data = []
test_data.append(test)
all_recs.append(eval('ItemItem', algo_ii, train, test))
all_recs.append(eval('ALS', algo_als, train, test))

In [32]:
all_recs.head(1000)



,item,score,user,rank,Algorithm
0,259,1.0,2,1,ALS
1,8533,1.0,2,2,ALS
2,13727,1.0,2,3,ALS
3,14620,1.0,2,4,ALS
4,0,1.0,2,5,ALS
...,...,...,...,...,...
995,15780,1.0,62,96,ALS
996,3828,1.0,62,97,ALS
997,3420,1.0,62,98,ALS
998,5869,1.0,62,99,ALS


In [34]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test)
results.head()

,,nrecs,ndcg
user,Algorithm,,
2,ALS,100.0,0.0
10,ALS,100.0,0.0
13,ALS,100.0,0.0
29,ALS,100.0,0.0
33,ALS,100.0,0.0
